In [3]:
import Auto_paint_self

In [4]:
CHM_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\CHM_PRE_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
CMO_RPH,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\CMORPH_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
ERA5_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\ERA5_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
GPM_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\GPM_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
GSMAP_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\GSMAP_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
MSWEP_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\MSWEP_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
PERSIANN_CDR_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\PERSIANN_CDR_PRE_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')

In [5]:
import numpy as np
vx=np.zeros((CMO_RPH.shape[0],CMO_RPH.shape[1],CMO_RPH.shape[2],6))
vx[:,:,:,0]=CMO_RPH
vx[:,:,:,1]=ERA5_PRE
vx[:,:,:,2]=GPM_PRE
vx[:,:,:,3]=GSMAP_PRE
vx[:,:,:,4]=MSWEP_PRE
vx[:,:,:,5]=PERSIANN_CDR_PRE
vx=np.nan_to_num(vx,nan=0)
vx=(vx-np.nanmean(vx,axis=0))/np.nanstd(vx,axis=0)

C:\Users\TBYC\AppData\Local\Temp\ipykernel_175264\3698730732.py:10: RuntimeWarning: invalid value encountered in divide
  vx=(vx-np.nanmean(vx,axis=0))/np.nanstd(vx,axis=0)


In [7]:
from keras.models import load_model
import tensorflow as tf
from scipy.stats import pearsonr
gpus = tf.config.list_physical_devices('GPU')
try:
    # 设置只使用 GPU 0
    tf.config.set_visible_devices(gpus[1], 'GPU')
    # 设置 GPU 0 的内存动态增长
    tf.config.experimental.set_memory_growth(gpus[1], True)
except RuntimeError as e:
    print(e)
index=int((1-0.2)*CHM_PRE.shape[0])
trainy=CHM_PRE[:index,:,:]
testy=CHM_PRE[index:,:,:]
trainx=vx[:index,:,:,:]
testx=vx[index:,:,:,:]
model=load_model(r'E:/xuanxuan/xuanxuan_unet')
predicty=model.predict(testx,batch_size=40)
predicty=predicty.reshape(predicty.shape[0],predicty.shape[1],predicty.shape[2])
predicty=predicty*np.nanstd(CHM_PRE,axis=0)+np.nanmean(CHM_PRE,axis=0)
r=np.zeros((testx.shape[1],testx.shape[2]))
p=np.zeros((testx.shape[1],testx.shape[2]))
for i in range(testx.shape[1]):
    for j in range(testx.shape[2]):
        if np.isnan(testy[:,i,j]).any() or np.isnan(predicty[:,i,j]).any():
            r[i,j],p[i,j]=np.nan,np.nan
        else:
            r[i,j],p[i,j] = pearsonr(predicty[:,i,j],testy[:,i,j])
print(np.nanmean(r))

41/41 [==============================] - 3s 25ms/step


F:\anaconda\envs\tensorflow\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\TBYC\AppData\Local\Temp\ipykernel_175264\24706048.py:20: RuntimeWarning: Mean of empty slice
  predicty=predicty*np.nanstd(CHM_PRE,axis=0)+np.nanmean(CHM_PRE,axis=0)


0.6151126967231928
